# 5T-OTA Example

## Python Setup

In [29]:
from pygmid import Lookup as lk
import numpy as np

nmos = lk('../mat/nfet_01v8_lvt.mat')
pmos = lk('../mat/pfet_01v8_lvt.mat')

## Input Parameters

In [ ]:
# Circuit Parameters
Vdd = 1.8
ugf = 50e6
Cload = 1e-12

# Device Parameters
## gm/ID
gm_ID_M12 = 20
gm_ID_M34 = 6
gm_ID_M56 = 10

## Lengths
L_M12 = 0.5
L_M34 = 1
L_M56 = 1

## Calculations

In [31]:
# Vdsat
Vdsat_M12 = 2 / gm_ID_M12
Vdsat_M34 = 2 / gm_ID_M34
Vdsat_M56 = 2 / gm_ID_M56

# Vgs
Vgs_M12 = nmos.lookupVGS(GM_ID=gm_ID_M12, VDS=Vdd/2, VSB=Vdsat_M56, L=L_M12)
Vgs_M34 = pmos.lookupVGS(GM_ID=gm_ID_M34, VSB=0, L=L_M34)
Vgs_M34 = pmos.lookupVGS(GM_ID=gm_ID_M34, VDS=Vgs_M34, VSB=0, L=L_M34)
Vgs_M56 = nmos.lookupVGS(GM_ID=gm_ID_M12, VDS=Vdsat_M56, VSB=0, L=L_M56)

# 'Optimize' Loop
Cself = 0
for k in range(20):
    # Input Transconductance (gm [S]) & Current (ID [A])
    gm_M12 = 2 * np.pi * (Cload + Cself) * ugf
    ID_M12 = ID_M34 = gm_M12 / gm_ID_M12

    # Load Transconductance (gm [S])
    gm_M34 = gm_ID_M34 * ID_M34

    # Input & Load Current Density (JD [µA/m])
    JD_M12 = nmos.lookup('ID_W', GM_ID=gm_ID_M12, VDS=Vdd/2, VSB=Vdsat_M56, L=L_M12)
    JD_M34 = pmos.lookup('ID_W', GM_ID=gm_ID_M34, VDS=Vgs_M34, VSB=0, L=L_M34)

    # Input & Load Width (W [µm])
    W_M12 = ID_M12 / JD_M12
    W_M34 = ID_M34 / JD_M34

    # Input & Load Drain Capacitance (CDD [F])
    Cdd_M12 = nmos.lookup('CDD_W', GM_ID=gm_ID_M12, VDS=Vdd/2, VSB=Vdsat_M56, L=L_M12) * W_M12
    Cdd_M34 = pmos.lookup('CDD_W', GM_ID=gm_ID_M34, VDS=Vgs_M34, VSB=0, L=L_M34) * W_M34
    Cself = (Cdd_M12 + Cdd_M34)

# Tail Current, Transconductance, Current Density & Width
ID_M56 = 2 * ID_M12
gm_M56 = gm_ID_M56 * ID_M56
JD_M56 = nmos.lookup('ID_W', GM_ID=gm_ID_M56, VDS=Vdsat_M56, VSB=0, L=L_M56)
W_M56 = ID_M56 / JD_M56

# Intrinsic Gain (gm/gds [V/V])
av_M12 = nmos.lookup('GM_GDS', GM_ID=gm_ID_M12, VDS=Vdd/2, VSB=Vdsat_M56, L=L_M12)
av_M34 = pmos.lookup('GM_GDS', GM_ID=gm_ID_M34, VDS=Vgs_M34, VSB=0, L=L_M34)
av_M56 = nmos.lookup('GM_GDS', GM_ID=gm_ID_M56, VDS=Vdsat_M56, VSB=0, L=L_M56)

# Output Conductance (gds [S])
gds_M12 = gm_M12 / av_M12
gds_M34 = gm_M34 / av_M34
gds_M56 = gm_M56 / av_M56

# Threshold Voltage (Vth [V])
Vth_M12 = nmos.lookup('VT', VGS=Vgs_M12, VDS=Vdd/2, VSB=Vdsat_M56, L=L_M12)
Vth_M34 = pmos.lookup('VT', VGS=Vgs_M34, VDS=Vgs_M34, VSB=0, L=L_M34)
Vth_M56 = nmos.lookup('VT', VGS=Vgs_M56, VDS=Vgs_M56, VSB=0, L=L_M56)

# Circuit Gain (A0 [V/V, dB])
A0_Mag = gm_M12 / (gds_M12 + gds_M34)
A0_dB = 20 * np.log10(A0_Mag)

# Circuit Consumption (IDD [A])
Idd = ID_M56

# Circuit Unity-Gain Frequency (UGF [Hz])
UGF = gm_M12 / (2 * np.pi * (Cload + Cself))

In [32]:
# Output
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+")
print(f"| {'Parameter':<13} | {'M1/2':<10} | {'M3/4':<10} | {'M5/6':<10} | {'Unit':<10} |")
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+")
print(f"| {'W':<13} | {W_M12:>10.3f} | {W_M34:>10.3f} | {W_M56:>10.3f} | {'µm':<10} |")
print(f"| {'L':<13} | {L_M12:>10.3f} | {L_M34:>10.3f} | {L_M56:>10.3f} | {'µm':<10} |")
print(f"| {'gm/ID':<13} | {gm_ID_M12:>10.3f} | {gm_ID_M34:>10.3f} | {gm_ID_M56:>10.3f} | {'S/A':<10} |")
print(f"| {'JD':<13} | {JD_M12/1e-6:>10.3f} | {JD_M34/1e-6:>10.3f} | {JD_M56/1e-6:>10.3f} | {'µA/µm':<10} |")
print(f"| {'gm/gds':<13} | {av_M12:>10.3f} | {av_M34:>10.3f} | {av_M56:>10.3f} | {'V/V':<10} |")
print(f"| {'gm':<13} | {gm_M12/1e-6:>10.3f} | {gm_M34/1e-6:>10.3f} | {gm_M56/1e-6:>10.3f} | {'µS':<10} |")
print(f"| {'ID':<13} | {ID_M12/1e-6:>10.3f} | {ID_M34/1e-6:>10.3f} | {ID_M56/1e-6:>10.3f} | {'µA':<10} |")
print(f"| {'gds':<13} | {gds_M12/1e-6:>10.3f} | {gds_M34/1e-6:>10.3f} | {gds_M56/1e-6:>10.3f} | {'µS':<10} |")
print(f"| {'Vth':<13} | {Vth_M12/1e-3:>10.3f} | {Vth_M34/1e-3:>10.3f} | {Vth_M56/1e-3:>10.3f} | {'mV':<10} |")
print(f"| {'VGS':<13} | {'-':>10} | {Vgs_M34/1e-3:>10.3f} | {Vgs_M56/1e-3:>10.3f} | {'mV':<10} |")
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "+")

print("\nCIRCUIT PARAMETERS")
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+")
print(f"| {'Parameter':<13} | {'Value':<10} | {'Unit':<10} |")
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+")
print(f"| {'A0_Mag':<13} | {A0_Mag:>10.3f} | {'V/V':<10} |")
print(f"| {'A0_dB':<13} | {A0_dB:>10.3f} | {'dB':<10} |")
print(f"| {'UGF':<13} | {UGF/1e6:>10.3f} | {'MHz':<10} |")
print(f"| {'IDD':<13} | {Idd/1e-6:>10.3f} | {'µA':<10} |")
print("+" + "-"*15 + "+" + "-"*12 + "+" + "-"*12 + "+")

+---------------+------------+------------+------------+------------+
| Parameter     | M1/2       | M3/4       | M5/6       | Unit       |
+---------------+------------+------------+------------+------------+
| W             |     16.383 |      5.217 |      7.991 | µm         |
| L             |      0.500 |      1.000 |      1.000 | µm         |
| gm/ID         |     20.000 |      6.000 |     10.000 | S/A        |
| JD            |      0.971 |      3.048 |      3.980 | µA/µm      |
| gm/gds        |     89.917 |     55.809 |     12.860 | V/V        |
| gm            |    318.044 |     95.413 |    318.044 | µS         |
| ID            |     15.902 |     15.902 |     31.804 | µA         |
| gds           |      3.537 |      1.710 |     24.730 | µS         |
| Vth           |    554.200 |    444.260 |    483.000 | mV         |
| VGS           |          - |    790.084 |    507.608 | mV         |
+---------------+------------+------------+------------+------------+

CIRCUIT PARAMETERS
